### Projeto 2-1 Visão Computacional
#### Elisa Malzoni

Esse projeto foi feito em etapas conforme as rúbricas e cada etapa pode ser vista abaixo.

In [1]:
%reset -f
%matplotlib inline

import cv2 as cv2
import numpy as np
import matplotlib.pyplot as plt

import time

## Aula 09 - Good features to Track

Estabilização parcial com os programas da aula 09.

In [2]:
captura = cv2.VideoCapture(0)

# Para não deixar encavalar os frames
captura.set(cv2.CAP_PROP_BUFFERSIZE, 1)

# Parametriza a funcao do OpenCV
dt_params = dict( maxCorners = 100,
           qualityLevel = 0.3,
           minDistance = 7,
           blockSize = 7 )

color = np.random.randint(0,255,(100,3))

# Parametriza o Lucas-Kanade
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))


ret, previous = captura.read()

previous_gray = cv2.cvtColor(previous, cv2.COLOR_RGB2GRAY)
p0 = cv2.goodFeaturesToTrack(previous_gray, mask = None, **dt_params)
mask = np.zeros_like(previous)

fomv = [0,0] # vetor do fluxo ótico médio acumulado 

while(1):
    
    ret, actual = captura.read()
    actual_gray = cv2.cvtColor(actual, cv2.COLOR_RGB2GRAY)
    
    # Calcula Fluxo Otico
    p1, st, err = cv2.calcOpticalFlowPyrLK(previous_gray, actual_gray, p0, None, **lk_params)
        
    # Seleciona somente os melhores pontos
    good_new = p1[st==1]
    good_old = p0[st==1]
       

    # Desenha as trilhas para cada ponto em p1 e p0
    for i,(new, old) in enumerate(zip(good_new, good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), [0,0,255], 2)
        actual = cv2.circle(actual,(a,b),5,color[i].tolist(),-1)
    

    # Calcula fluxo otico medio
    sub = np.subtract(good_new, good_old)
    fom = np.mean(sub, axis=0) 
    fom = (-1)*fom
    fomv += fom
    
    frame = cv2.add(actual, mask)
    framebw = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    rows, cols = framebw.shape
    
    # componentes do vertor inverso do fluxo ótico
    tx = (int(fomv[0]))
    ty = (int(fomv[1]))
    
    # Matriz de translação
    M = np.float32([[1,0,tx],
                    [0,1,ty]])
    
    # frame transladado
    framet = cv2.warpAffine(frame, M,(cols,rows))
    
    # mostra frame transladado
    cv2.imshow("Video", framet)
    
    # Atualiza a imagem anterior com a imagem atual e copia os pontos.
    previous_gray = actual_gray.copy()
    p0 = good_new.reshape(-1,1,2)
    
    
    # Pressione ESC para sair do loop
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    time.sleep(0.1)

captura.release()
cv2.destroyAllWindows()

In [3]:
captura.release()

## Dense Optical flow

Dense Optical Flow de uma janela no centro da imagem, quadrada com lado $l$.

In [7]:
captura = cv2.VideoCapture(0)

# Para não deixar encavalar os frames
captura.set(cv2.CAP_PROP_BUFFERSIZE, 1)

ret, previous = captura.read() # captura imagem da câmera
previous = cv2.flip(previous,1) # flipa horizontalmente a imagem

previous_gray = cv2.cvtColor(previous, cv2.COLOR_RGB2GRAY) #converte a imagem capturada para escala de cinza

# dimensões do frame
w = previous.shape[0]
h = previous.shape[1]

fomv = [0,0] #vetor do fluxo ótico

while(1):
    
    ret, actual = captura.read()
    actual = cv2.flip(actual,1)
    actual_gray = cv2.cvtColor(actual, cv2.COLOR_RGB2GRAY)
    
    # Calcula Fluxo Otico
    l = 120 # tamanho do lado da janela no centro, quadrada, onde o fluxo ótico será calculado
    previous_gray_center = previous_gray[int((w/2)-(l/2)+1):int((w/2)+(l/2)+1), int((h/2)-(l/2)+1):int((h/2)+(l/2)+1)]#slicing para pegar o centro da imagem
    actual_gray_center = actual_gray[int((w/2)-(l/2)+1):int((w/2)+(l/2)+1), int((h/2)-(l/2)+1):int((h/2)+(l/2)+1)]    
    flow = cv2.calcOpticalFlowFarneback(previous_gray_center, actual_gray_center,None, 0.5, 3, 15, 3, 5, 1.2, 0)       
    
    # Calcula fluxo ótico médio em x e em y
    xmean = (np.mean(flow[:,:,0]))
    ymean = (np.mean(flow[:,:,1]))
    fom = np.array([xmean, ymean])
    fom *= -1
    fomv += fom # acumula o fluxo ótico médio
         
    # converte o frame atual para escala de cinza
    framebw = cv2.cvtColor(actual, cv2.COLOR_RGB2GRAY) 
    # dimensões do frame convertido
    rows, cols = framebw.shape 
    
    # componentes do vertor inverso do fluxo ótico
    tx = (int(fomv[0]))
    ty = (int(fomv[1]))
    
    # matriz de translação
    M = np.float32([[1,0,tx],
                    [0,1,ty]])

    # translada o frame
    framet = cv2.warpAffine(actual, M,(cols,rows))

    # mostra o frame
    cv2.imshow("Video", framet)
    
    # mostra a janela central onde o fluxo está sendo calculado
#     cv2.imshow("centro", actual_gray_center)
    
    # Atualiza a imagem anterior com a imagem atual e copia os pontos.
    previous_gray = actual_gray
    
    # Pressione ESC para sair do loop
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    time.sleep(0.1)

captura.release()
cv2.destroyAllWindows()

In [8]:
captura.release()


Complementando a célula acima porém com compensação das faixas pretas nos cantos da imagem com algum limite.

In [9]:
captura = cv2.VideoCapture(0)

# Para não deixar encavalar os frames
captura.set(cv2.CAP_PROP_BUFFERSIZE, 1)

ret, previous = captura.read()
previous = cv2.flip(previous,1)

previous_gray = cv2.cvtColor(previous, cv2.COLOR_RGB2GRAY)
hsv = np.zeros_like(previous)

# dimensões do frame
w = previous.shape[0]
h = previous.shape[1]

ratio = w/h

fomv = [0,0] #vetor do fluxo ótico acumulado

while(1):
    
    ret, actual = captura.read()
    actual = cv2.flip(actual,1)
    actual_gray = cv2.cvtColor(actual, cv2.COLOR_RGB2GRAY)
    
    # Calcula Fluxo Otico
    l = 120 # tamanho do lado da janela no centro, quadrada, onde o fluxo será calculado
    previous_gray_center = previous_gray[int((w/2)-(l/2)+1):int((w/2)+(l/2)+1), int((h/2)-(l/2)+1):int((h/2)+(l/2)+1)]#slicing para pegar o centro da imagem
    actual_gray_center = actual_gray[int((w/2)-(l/2)+1):int((w/2)+(l/2)+1), int((h/2)-(l/2)+1):int((h/2)+(l/2)+1)]
    flow = cv2.calcOpticalFlowFarneback(previous_gray_center, actual_gray_center,None, 0.5, 3, 15, 3, 5, 1.2, 0)       
    
    # Calcula fluxo ótico médio em x e em y
    xmean = (np.mean(flow[:,:,0]))
    ymean = (np.mean(flow[:,:,1]))
    fom = np.array([xmean, ymean])
    # inverte vetor do Fluxo ótico
    fom *= -1
    fomv += fom
         
    
    framebw = cv2.cvtColor(actual, cv2.COLOR_RGB2GRAY)
    rows, cols = framebw.shape
    
    # componentes do inverso do fluxo ótico
    tx = (int(fomv[0]))
    ty = (int(fomv[1]))
    
    # Matriz de Translação
    M = np.float32([[1,0,tx],
                    [0,1,ty]])
    
    #frame transladado
    framet = cv2.warpAffine(actual, M,(cols,rows))

    #scaling
    #dimensões imagem cortada
    wt = w - abs(tx)
    ht = h - abs(ty)
    
    # Proporção da imagem cortada
    ratio_cut = wt/ht
    
    # Assumir que as dimensões da imagem permanecerão as mesmas
    wtc = wt
    htc = ht
    
    if ratio > ratio_cut:
        #corta em y
        htc = int((h*wt)/w)

    else:
        #corta em x
        wtc = int((ht*w)/h)
    
    # Cálculo da origem após cortar a imagem
    if tx < 0:
        origin_x_cut = w - wtc + tx
    else:
        origin_x_cut = w - wtc

    if ty < 0:
        origin_y_cut = h - htc + ty
    else:
        origin_y_cut = h - htc
        
    new_ratio_cut = wtc/htc
    
    
    # slicing da imagem com o mesmo ratio da câmera
    framet_cut  = framet[origin_x_cut:(origin_x_cut+wtc), origin_y_cut:(origin_y_cut + htc)]

    
    # Matriz de translação para trazer a imagem de volta ao centro
    M2 = np.float32([[1,0,int(-origin_x_cut/2)],
                    [0,1,int(-origin_y_cut/2)]])
    
    rows2, cols2, a = framet_cut.shape
    
    # Frame transladado da imagem cortada
    framet_cut = cv2.warpAffine(actual, M2,(cols2,rows2))

    # redimensionamento do frame acima
    frame_ok = cv2.resize(framet_cut,(h,w), interpolation = cv2.INTER_CUBIC)
            
    # mostra frame redimensionado    
    cv2.imshow("Estabilizado", frame_ok)

    # mostra frame original sem tratamentos
#     cv2.imshow("a", actual) 

    # mostra a janela central onde o fluxo está sendo calculado
#     cv2.imshow("centro", actual_gray_center)
    
    # Atualiza a imagem anterior com a imagem atual e copia os pontos.
    previous_gray = actual_gray
        
    # Pressione ESC para sair do loop
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    time.sleep(0.1)

captura.release()
cv2.destroyAllWindows()

In [6]:
captura.release()
